# Sales Forecasting & Performance Analysis

## Load Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

df = pd.read_csv('sales_data_1000_rows.csv')
df

,OrderID,OrderDate,Product,Category,Region,Quantity,UnitPrice,TotalSales,CustomerType,PaymentMode
0,10000,2024-06-06,Mouse,Accessories,West,4,1360,5440,Returning,UPI
1,10001,2023-05-10,Laptop,Electronics,South,3,77320,231960,New,UPI
2,10002,2024-06-24,Mobile,Electronics,South,2,37694,75388,Returning,Credit Card
3,10003,2024-06-03,Camera,Electronics,West,3,44631,133893,New,UPI
4,10004,2023-02-23,Laptop,Electronics,West,3,67721,203163,Returning,Debit Card
...,...,...,...,...,...,...,...,...,...,...
995,10995,2023-02-27,Smartwatch,Wearables,South,4,11256,45024,Returning,Cash on Delivery
996,10996,2024-07-04,Laptop,Electronics,East,4,36878,147512,Returning,Cash on Delivery
997,10997,2024-10-18,Monitor,Electronics,North,5,57049,285245,New,Cash on Delivery
998,10998,2023-03-29,Mouse,Accessories,South,2,74294,148588,Returning,UPI


## Inspect the Dataset

In [2]:
print(df.head())

   OrderID   OrderDate Product     Category Region  Quantity  UnitPrice  \
0    10000  2024-06-06   Mouse  Accessories   West         4       1360   
1    10001  2023-05-10  Laptop  Electronics  South         3      77320   
2    10002  2024-06-24  Mobile  Electronics  South         2      37694   
3    10003  2024-06-03  Camera  Electronics   West         3      44631   
4    10004  2023-02-23  Laptop  Electronics   West         3      67721   

   TotalSales CustomerType  PaymentMode  
0        5440    Returning          UPI  
1      231960          New          UPI  
2       75388    Returning  Credit Card  
3      133893          New          UPI  
4      203163    Returning   Debit Card  


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   OrderID       1000 non-null   int64 
 1   OrderDate     1000 non-null   object
 2   Product       1000 non-null   object
 3   Category      1000 non-null   object
 4   Region        1000 non-null   object
 5   Quantity      1000 non-null   int64 
 6   UnitPrice     1000 non-null   int64 
 7   TotalSales    1000 non-null   int64 
 8   CustomerType  1000 non-null   object
 9   PaymentMode   1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.3+ KB
None


In [4]:
print(df.describe())

            OrderID     Quantity     UnitPrice     TotalSales
count   1000.000000  1000.000000   1000.000000    1000.000000
mean   10499.500000     2.950000  38917.110000  112242.604000
std      288.819436     1.442769  22431.323548   88942.063882
min    10000.000000     1.000000    560.000000     560.000000
25%    10249.750000     2.000000  20472.000000   42718.500000
50%    10499.500000     3.000000  38720.000000   85135.000000
75%    10749.250000     4.000000  57054.250000  161757.500000
max    10999.000000     5.000000  79773.000000  396285.000000


## Data Cleaning

In [5]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])
df = df.dropna()
df = df[df['TotalSales'] > 0]

In [6]:
print(df)

     OrderID  OrderDate     Product     Category Region  Quantity  UnitPrice  \
0      10000 2024-06-06       Mouse  Accessories   West         4       1360   
1      10001 2023-05-10      Laptop  Electronics  South         3      77320   
2      10002 2024-06-24      Mobile  Electronics  South         2      37694   
3      10003 2024-06-03      Camera  Electronics   West         3      44631   
4      10004 2023-02-23      Laptop  Electronics   West         3      67721   
..       ...        ...         ...          ...    ...       ...        ...   
995    10995 2023-02-27  Smartwatch    Wearables  South         4      11256   
996    10996 2024-07-04      Laptop  Electronics   East         4      36878   
997    10997 2024-10-18     Monitor  Electronics  North         5      57049   
998    10998 2023-03-29       Mouse  Accessories  South         2      74294   
999    10999 2024-01-12    Keyboard  Accessories   East         4      45613   

     TotalSales CustomerType       Paym

## Exploratory Data Analysis

### Monthly Sales Trend

In [8]:
monthly = df.resample('M', on='OrderDate')['TotalSales'].sum().reset_index()
print(monthly.head())

   OrderDate  TotalSales
0 2023-01-31     4657108
1 2023-02-28     5000497
2 2023-03-31     5335284
3 2023-04-30     3262588
4 2023-05-31     5501254


C:\Users\C.POORNA\AppData\Local\Temp\ipykernel_17560\2854264542.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df.resample('M', on='OrderDate')['TotalSales'].sum().reset_index()


### Sales by Region

In [9]:
print(df.groupby('Region')['TotalSales'].sum())

Region
East     25435107
North    31277720
South    28763976
West     26765801
Name: TotalSales, dtype: int64


### Sales by Category

In [10]:
print(df.groupby('Category')['TotalSales'].sum())

Category
Accessories    32524473
Electronics    55774812
Office         13101865
Wearables      10841454
Name: TotalSales, dtype: int64


## Forecast Next 3 Months Using Prophet

### Prepare forecasting data

In [11]:
ts = monthly.rename(columns={'OrderDate': 'ds', 'TotalSales': 'y'})

In [12]:
print(ts)

           ds        y
0  2023-01-31  4657108
1  2023-02-28  5000497
2  2023-03-31  5335284
3  2023-04-30  3262588
4  2023-05-31  5501254
5  2023-06-30  5713606
6  2023-07-31  3588808
7  2023-08-31  5142154
8  2023-09-30  2700566
9  2023-10-31  3663010
10 2023-11-30  4310636
11 2023-12-31  4736201
12 2024-01-31  5961344
13 2024-02-29  4528594
14 2024-03-31  3298951
15 2024-04-30  5193887
16 2024-05-31  4100227
17 2024-06-30  5832399
18 2024-07-31  4952357
19 2024-08-31  4408314
20 2024-09-30  4927779
21 2024-10-31  4786715
22 2024-11-30  4801778
23 2024-12-31  5838547


### Build the model

In [14]:
from prophet import Prophet

model = Prophet()
model.fit(ts)

19:34:15 - cmdstanpy - INFO - Chain [1] start processing
19:34:15 - cmdstanpy - INFO - Chain [1] done processing


### Predict 90 days (3 months)

In [15]:
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)

In [17]:
print(future)

            ds
0   2023-01-31
1   2023-02-28
2   2023-03-31
3   2023-04-30
4   2023-05-31
..         ...
109 2025-03-27
110 2025-03-28
111 2025-03-29
112 2025-03-30
113 2025-03-31

[114 rows x 1 columns]


In [18]:
print(forecast)

            ds         trend    yhat_lower    yhat_upper   trend_lower  \
0   2023-01-31  4.437300e+06  3.382474e+06  5.495022e+06  4.437300e+06   
1   2023-02-28  4.456399e+06  3.361766e+06  5.573786e+06  4.456399e+06   
2   2023-03-31  4.477545e+06  3.356513e+06  5.618569e+06  4.477545e+06   
3   2023-04-30  4.498009e+06  3.460609e+06  5.631436e+06  4.498009e+06   
4   2023-05-31  4.519155e+06  3.457886e+06  5.666500e+06  4.519155e+06   
..         ...           ...           ...           ...           ...   
109 2025-03-27  4.973450e+06  3.851046e+06  6.079629e+06  4.973450e+06   
110 2025-03-28  4.974132e+06  3.812506e+06  5.995690e+06  4.974132e+06   
111 2025-03-29  4.974814e+06  3.830227e+06  6.075161e+06  4.974814e+06   
112 2025-03-30  4.975496e+06  3.967418e+06  6.174010e+06  4.975496e+06   
113 2025-03-31  4.976178e+06  3.910341e+06  6.051278e+06  4.976178e+06   

      trend_upper  additive_terms  additive_terms_lower  additive_terms_upper  \
0    4.437300e+06             

### Save forecast file

In [19]:
forecast[['ds','yhat','yhat_lower','yhat_upper']].to_csv("sales_forecast.csv", index=False)

In [20]:
print(forecast)

            ds         trend    yhat_lower    yhat_upper   trend_lower  \
0   2023-01-31  4.437300e+06  3.382474e+06  5.495022e+06  4.437300e+06   
1   2023-02-28  4.456399e+06  3.361766e+06  5.573786e+06  4.456399e+06   
2   2023-03-31  4.477545e+06  3.356513e+06  5.618569e+06  4.477545e+06   
3   2023-04-30  4.498009e+06  3.460609e+06  5.631436e+06  4.498009e+06   
4   2023-05-31  4.519155e+06  3.457886e+06  5.666500e+06  4.519155e+06   
..         ...           ...           ...           ...           ...   
109 2025-03-27  4.973450e+06  3.851046e+06  6.079629e+06  4.973450e+06   
110 2025-03-28  4.974132e+06  3.812506e+06  5.995690e+06  4.974132e+06   
111 2025-03-29  4.974814e+06  3.830227e+06  6.075161e+06  4.974814e+06   
112 2025-03-30  4.975496e+06  3.967418e+06  6.174010e+06  4.975496e+06   
113 2025-03-31  4.976178e+06  3.910341e+06  6.051278e+06  4.976178e+06   

      trend_upper  additive_terms  additive_terms_lower  additive_terms_upper  \
0    4.437300e+06             